In [1]:
import os
import re
import json
import requests
import datetime
import subprocess
import xml.etree.ElementTree as ET

#默认参数
default_config = {
    "channelid_youtube": {
        "YouTube": {
            "update_size": 1,
            "id": "UCBR8-60-B28hp2BmDPdntcQ",
            "title": "YouTube",
            "quality": "720",
            "last_size": 1,
            "media": "m4a"
        }
    }
}

In [2]:
#日志模块
def write_log(log):
    # 获取当前的具体时间
    current_time = datetime.datetime.now()
    # 格式化输出，只保留年月日时分秒
    formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
    # 打开文件，并读取原有内容
    try:
        with open("log.txt", "r") as file:
            contents = file.read()
    except FileNotFoundError:
        contents = ""
    # 将新的日志内容添加在原有内容之前
    new_contents = formatted_time + " " + log + "\n" + contents
    # 将新的日志内容写入文件
    with open("log.txt", "w") as file:
        file.write(new_contents)
    # 打印当前的具体时间（精确到秒）
    print(formatted_time, log)

In [3]:
# 安装库模块
def library_install(library):
    # 检查库是否已安装
    def is_library_installed():
        try:
            result = subprocess.run([library , '--version'], capture_output=True, text=True)
            if result.returncode == 0:
                return True
            else:
                return False
        except FileNotFoundError:
            return False
    # 如果库未安装，则尝试安装
    def install_library():
        try:
            result = subprocess.run(['pip', 'install', library], capture_output=True, text=True)
            if result.returncode == 0:
                return True
            else:
                return False
        except FileNotFoundError:
            return False
    # 如果库已安装，则尝试更新
    def update_library():
        try:
            result = subprocess.run(['pip', 'install', '--upgrade', library], capture_output=True, text=True)
            if result.returncode == 0:
                return True
            else:
                return False
        except FileNotFoundError:
            return False
    # 检查库是否已安装
    if is_library_installed():
        write_log(library + "已安装")
    else:
        write_log(library + "未安装")
    # 如果库已安装，则尝试更新
    if is_library_installed():
        if update_library():
            write_log(library + "更新成功")
        else:
            write_log(library + "更新失败")
    else:  # 如果库未安装，则尝试安装
        if install_library():
            write_log(library + "安装成功")
        else:
            write_log(library + "安装失败")

In [4]:
# 安装/更新yt-dlp
library_install("yt-dlp")
# 安装/更新pytube
library_install("pytube")

2023-08-09 03:32:03 yt-dlp已安装
2023-08-09 03:32:07 yt-dlp更新成功
2023-08-09 03:32:07 pytube已安装
2023-08-09 03:32:09 pytube更新成功


In [5]:
# 检查当前文件夹中是否存在config.json文件
if not os.path.exists('config.json'):
    # 如果文件不存在，创建并写入默认字典
    with open('config.json', 'w') as file:
        json.dump(default_config, file, indent=4)
    write_log("不存在配置文件，已新建，默认频道")
    config = default_config
else:
    # 如果文件存在，读取字典并保存到config变量中
    with open('config.json', 'r') as file:
        config = json.load(file)
    write_log("已读取配置文件")

2023-08-09 03:32:09 已读取配置文件


In [6]:
# 从配置文件中获取YouTube的频道
channelid_youtube = config.get("channelid_youtube")
if channelid_youtube is not None:
    write_log("已读取youtube频道信息")
else:
    write_log("youtube频道信息不存在")
# 从配置文件中获取bilibili的频道
channelid_bilibili = config.get("channelid_bilibili")
if channelid_bilibili is not None:
    write_log("已读取bilibili频道信息")
else:
    write_log("bilibili频道信息不存在")

2023-08-09 03:32:09 已读取youtube频道信息
2023-08-09 03:32:09 bilibili频道信息不存在


In [7]:
# 构建文件夹channel_id
folder_path_channel_ids = os.path.join(os.getcwd(), "channel_id")
if not os.path.exists(folder_path_channel_ids):  # 判断文件夹是否存在
    os.makedirs(folder_path_channel_ids)  # 创建文件夹
    write_log("文件夹channel_id创建成功")

In [8]:
# 读取youtube频道的id
if channelid_youtube is not None:
    channelid_youtube_ids = dict({channel["id"]: channel["title"] for channel in channelid_youtube.values()})
    write_log("读取youtube频道的id成功")
else:
    channelid_youtube_ids = None
# 读取bilibili频道的id
if channelid_bilibili is not None:
    channelid_bilibili_ids = [channelid_bilibili[key]['id'] for key in channelid_bilibili]
    write_log("读取bilibili频道的id成功")
else:
    channelid_bilibili_ids = None

2023-08-09 03:32:09 读取youtube频道的id成功


In [10]:
# 更新Youtube频道xml
for key, value in channelid_youtube_ids.items():
    # 构建 URL
    url = f"https://www.youtube.com/feeds/videos.xml?channel_id={key}"
    # 发送请求并获取响应内容
    response = requests.get(url)
    content = response.text
    # 判断频道id是否正确
    pattern_youtube404 = r"Error 404"  # 设置要匹配的正则表达式模式
    pattern_youtube_vary = r'([0-9]{4}-[0-9]{2}-[0-9]{2}T[0-2][0-9]:[0-6][0-9]:[0-6][0-9]\+00:00)?(starRating count="[0-9]*")?(statistics views="[0-9]*")?(<id>yt:channel:(UC.{22})?</id>)?(<yt:channelId>(UC.{22})?</yt:channelId>)?'
    if not re.search(pattern_youtube404, content):
        content = re.sub(pattern_youtube_vary, '', content)
        # 构建文件路径
        file_path = os.path.join(folder_path_channel_ids, f"{key}.txt")
        # 构建文件路径
        with open(file_path, "w", encoding="utf-8") as file:
            file.write(content)
            write_log(f"YouTube频道 {value} 已更新")
        # 构建频道文件夹
        folder_path_channel_id = os.path.join(os.getcwd(), key)
        if not os.path.exists(folder_path_channel_id):  # 判断文件夹是否存在
            os.makedirs(folder_path_channel_id)  # 创建文件夹
            write_log(f"文件夹{key}创建成功")
    else:
        write_log(f"YouTube频道 {value} ID不正确无法获取")
        del channelid_youtube_ids[key]  # 删除错误ID

2023-08-09 03:47:29 YouTube频道 YouTube 已更新
2023-08-09 03:47:29 YouTube频道 科学火箭叔 已更新
2023-08-09 03:47:29 YouTube频道 stone pistol 已更新
2023-08-09 03:47:30 YouTube频道 奇异博士说 已更新
2023-08-09 03:47:30 YouTube频道 科技美学 已更新
2023-08-09 03:47:30 YouTube频道 stone记 已更新
2023-08-09 03:47:31 YouTube频道 Linvo說宇宙 已更新
2023-08-09 03:47:31 YouTube频道 四季妈妈—小龙 已更新
2023-08-09 03:47:31 YouTube频道 Discovery With Andy 已更新
2023-08-09 03:47:32 YouTube频道 冲浪普拉斯 已更新
2023-08-09 03:47:32 YouTube频道 老高與小茉 Mr & Mrs Gao 已更新
2023-08-09 03:47:33 YouTube频道 脑洞乌托邦 已更新
2023-08-09 03:47:33 YouTube频道 啾啾鞋 已更新
2023-08-09 03:47:33 YouTube频道 幻海航行--science fiction 已更新
2023-08-09 03:47:34 YouTube频道 红酒汤姆一世 已更新
2023-08-09 03:47:34 YouTube频道 X調查 已更新
2023-08-09 03:47:34 YouTube频道 海伦子Hellen 已更新
2023-08-09 03:47:35 YouTube频道 你可敢信 已更新
2023-08-09 03:47:35 YouTube频道 曉涵哥來了 已更新
2023-08-09 03:47:35 YouTube频道 自说自话的总裁 已更新
2023-08-09 03:47:35 YouTube频道 林亦LYi 已更新
2023-08-09 03:47:36 YouTube频道 小乌副频道 已更新


In [ ]:
def read_text_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
        return content
    except FileNotFoundError:
        print(f"File '{file_path}' not found.")
        return None

# 调用read_text_file函数来读取文件内容
file_path = 'channel_id/UCghLs6s95LrBWOdlZUCH4qw.txt'
text_content = read_text_file(file_path)

if text_content:
    print(text_content)

<?xml version="1.0" encoding="UTF-8"?>
<feed xmlns:yt="http://www.youtube.com/xml/schemas/2015" xmlns:media="http://search.yahoo.com/mrss/" xmlns="http://www.w3.org/2005/Atom">
 <link rel="self" href="http://www.youtube.com/feeds/videos.xml?channel_id=UCghLs6s95LrBWOdlZUCH4qw"/>
 <id>yt:channel:</id>
 <yt:channelId></yt:channelId>
 <title>stone记</title>
 <link rel="alternate" href="https://www.youtube.com/channel/UCghLs6s95LrBWOdlZUCH4qw"/>
 <author>
  <name>stone记</name>
  <uri>https://www.youtube.com/channel/UCghLs6s95LrBWOdlZUCH4qw</uri>
 </author>
 <published>2018-03-31T05:35:30+00:00</published>
 <entry>
  <id>yt:video:1M9DQXqv5ho</id>
  <yt:videoId>1M9DQXqv5ho</yt:videoId>
  <yt:channelId>UCghLs6s95LrBWOdlZUCH4qw</yt:channelId>
  <title>中国留学生在英国涂鸦24字”社会主义核心价值观”（20230807第6221期）</title>
  <link rel="alternate" href="https://www.youtube.com/watch?v=1M9DQXqv5ho"/>
  <author>
   <name>stone记</name>
   <uri>https://www.youtube.com/channel/UCghLs6s95LrBWOdlZUCH4qw</uri>
  </author>
  <p

In [ ]:
def youtube_element(channelid_youtube_id , channelid_youtube_name):
    if os.path.exists(f"channel_id/{channelid_youtube_id}.txt"):
        # 如果文件存在，读取字典并保存到channelid_youtube_xml变量中
        with open(f"channel_id/{channelid_youtube_id}.txt", 'r') as file:
            channelid_youtube_root = ET.fromstring(file)
            for entry in channelid_youtube_root.findall('.//{http://www.w3.org/2005/Atom}entry'):
                channelid_youtube_title = entry.find('{http://search.yahoo.com/mrss/}group/{http://search.yahoo.com/mrss/}title').text
                print(f"标题：{channelid_youtube_title}")
    else:
        # 如果文件不存在，删除错误ID
        del channelid_youtube_ids[channelid_youtube_id]
        write_log(f"YouTube频道 {channelid_youtube_name} XML文件不存在")


youtube_element("UCghLs6s95LrBWOdlZUCH4qw","stone记")


TypeError: a bytes-like object is required, not '_io.TextIOWrapper'